In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import csv
import pandas as pd

from datetime import datetime
import datetime as dt
from pytz import timezone

import json
from geojson import Feature, FeatureCollection, Point

import boto3

In [ ]:
##Importing Last 20wk sell-thru data and creating sell-thru dataframe

In [ ]:
sell_thru_df = pd.read_csv('Resources/for_sell_thru_update/sell_thru_data_to_be_updated_manually.csv') 
sell_thru_df = sell_thru_df.fillna(0)

In [ ]:
#Adding columns
sell_thru_df = sell_thru_df.rename(columns={'Store#': 'Location_Number'})

sell_thru_df["Last_Week_ST"] = sell_thru_df["Wk_1"]
sell_thru_df["Last_5wk_Ave_ST"] = (sell_thru_df["Wk_1"] + sell_thru_df["Wk_2"] + sell_thru_df["Wk_3"] + sell_thru_df["Wk_4"] + sell_thru_df["Wk_5"]) / 5
sell_thru_df["Last_10wk_Ave_ST"] = (sell_thru_df["Wk_1"] + sell_thru_df["Wk_2"] + sell_thru_df["Wk_3"] + sell_thru_df["Wk_4"] + sell_thru_df["Wk_5"] + sell_thru_df["Wk_6"] + sell_thru_df["Wk_7"] + sell_thru_df["Wk_8"] + sell_thru_df["Wk_9"] + sell_thru_df["Wk_10"]) / 10
sell_thru_df["Last_20wk_Ave_ST"] = (sell_thru_df["Wk_1"] + sell_thru_df["Wk_2"] + sell_thru_df["Wk_3"] + sell_thru_df["Wk_4"] + sell_thru_df["Wk_5"] + sell_thru_df["Wk_6"] + sell_thru_df["Wk_7"] + sell_thru_df["Wk_8"] + sell_thru_df["Wk_9"] + sell_thru_df["Wk_10"] + sell_thru_df["Wk_11"] + sell_thru_df["Wk_12"] + sell_thru_df["Wk_13"] + sell_thru_df["Wk_14"] + sell_thru_df["Wk_15"] + sell_thru_df["Wk_16"] + sell_thru_df["Wk_17"] + sell_thru_df["Wk_18"] + sell_thru_df["Wk_19"] + sell_thru_df["Wk_20"]) / 20

Last_Week_Total = sell_thru_df['Last_Week_ST'].sum()
Last_5Wk_Ave_Total = sell_thru_df['Last_5wk_Ave_ST'].sum()
Last_10Wk_Ave_Total = sell_thru_df['Last_10wk_Ave_ST'].sum()
Last_20Wk_Ave_Total = sell_thru_df['Last_20wk_Ave_ST'].sum()

sell_thru_df["Last_Week_Share"] = sell_thru_df["Last_Week_ST"] / Last_Week_Total * 100
sell_thru_df["Last_5Wk_Share"] = sell_thru_df["Last_5wk_Ave_ST"] / Last_5Wk_Ave_Total * 100
sell_thru_df["Last_10Wk_Share"] = sell_thru_df["Last_10wk_Ave_ST"] / Last_10Wk_Ave_Total * 100
sell_thru_df["Last_20Wk_Share"] = sell_thru_df["Last_20wk_Ave_ST"] / Last_20Wk_Ave_Total * 100

sell_thru_df["Last_5wk_Performance"] = sell_thru_df["Last_5Wk_Share"] / sell_thru_df["Last_20Wk_Share"] * 100
sell_thru_df = sell_thru_df.fillna(0)

In [ ]:
# Making time format
year_format = "%Y"

# Getting the current date & time
now_pst = datetime.now(timezone('US/Pacific'))
now_pst_last_week = now_pst - dt.timedelta(days=7)
tomorrow_pst = now_pst + dt.timedelta(days=1)
tomorrow_pst_last_week = tomorrow_pst - dt.timedelta(days=7)

In [ ]:
# Getting ISO weeknumber and convert the format
week_data = tomorrow_pst_last_week.isocalendar()[1]
year_data = now_pst_last_week.strftime(year_format)

if week_data < 10 :
    last_week = year_data + "w" + "0" + str(week_data)
else: 
    last_week = year_data + "w" + str(week_data)
    
last_week

In [ ]:
sell_thru_df["Last_Wk"] = last_week

In [ ]:
sell_thru_df = sell_thru_df[['Location_Number', 'Last_Week_ST', 'Last_5wk_Ave_ST', 'Last_10wk_Ave_ST', 'Last_20wk_Ave_ST', 'Last_Week_Share', 'Last_5Wk_Share', 'Last_10Wk_Share', 'Last_20Wk_Share', 'Last_5wk_Performance', 'Last_Wk']]

In [ ]:
#round the numbers in sell_thru_df
sell_thru_df = sell_thru_df.round({'Last_Week_ST': 0, 'Last_5wk_Ave_ST': 1, 'Last_10wk_Ave_ST': 1, 'Last_20wk_Ave_ST': 1, 'Last_Week_Share': 3, 'Last_5Wk_Share': 3, 'Last_10Wk_Share': 3, 'Last_20Wk_Share': 3, 'Last_5wk_Performance': 1})

In [ ]:
#Converting "Last_Week_ST" and "Last_5wk_Performance" into Integer
sell_thru_df.Last_Week_ST = sell_thru_df.Last_Week_ST.astype(int)
sell_thru_df.Last_5wk_Performance = sell_thru_df.Last_5wk_Performance.astype(int)

In [ ]:
sell_thru_df.head()

In [ ]:
##Importing Premium 10wk sell-thru data and adding it to sell-thru dataframe

In [ ]:
premium_sell_thru_df = pd.read_csv('Resources/for_sell_thru_update/premium_sell_thru_data_to_be_updated_manually.csv') 
premium_sell_thru_df = premium_sell_thru_df.fillna(0)

In [ ]:
#Adding columns
premium_sell_thru_df = premium_sell_thru_df.rename(columns={'Store#': 'Location_Number'})

premium_sell_thru_df["Premium_Last_10wk_Ave_ST"] = (premium_sell_thru_df["Wk_1"] + premium_sell_thru_df["Wk_2"] + premium_sell_thru_df["Wk_3"] + premium_sell_thru_df["Wk_4"] + premium_sell_thru_df["Wk_5"] + premium_sell_thru_df["Wk_6"] + premium_sell_thru_df["Wk_7"] + premium_sell_thru_df["Wk_8"] + premium_sell_thru_df["Wk_9"] + premium_sell_thru_df["Wk_10"]) / 10

premium_10wk_total = premium_sell_thru_df["Premium_Last_10wk_Ave_ST"].sum()

premium_sell_thru_df["Premium_10wk_Share"] = premium_sell_thru_df["Premium_Last_10wk_Ave_ST"] / premium_10wk_total * 100


In [ ]:
# Limiting only Location_Number, Last_10wk_AveST and Share
premium_sell_thru_df = premium_sell_thru_df[['Location_Number', 'Premium_Last_10wk_Ave_ST', 'Premium_10wk_Share']]
premium_sell_thru_df = premium_sell_thru_df.round({'Premium_Last_10wk_Ave_ST': 2, 'Premium_10wk_Share': 3})

In [ ]:
# Converting data type of Location_Number to str
premium_sell_thru_df.Location_Number = premium_sell_thru_df.Location_Number.astype(str)
sell_thru_df.Location_Number = sell_thru_df.Location_Number.astype(str)

In [ ]:
sell_thru_df = sell_thru_df.merge(premium_sell_thru_df, on='Location_Number', how='left')
sell_thru_df = sell_thru_df.fillna(0)

In [ ]:
sell_thru_df.head()

In [ ]:
##Importing Best Buy Geo Store List (Geo Data needs to be updated separately once Master Store List is updated )

In [ ]:
#Importing Store List with Geo Data
BBY_Store_List_GeoData_df = pd.read_csv('Resources/for_store_list_update/BestBuy_Store_List_GeoData.csv') 

In [ ]:
#Coverting Location_Number to String
BBY_Store_List_GeoData_df.Location_Number = BBY_Store_List_GeoData_df.Location_Number.astype(str)
sell_thru_df.Location_Number = sell_thru_df.Location_Number.astype(str)

In [ ]:
#Left Merge the df
BBY_Store_List_GeoData_Sell_thru_df = BBY_Store_List_GeoData_df.merge(sell_thru_df, on='Location_Number', how='left')

In [ ]:
BBY_Store_List_GeoData_Sell_thru_df.head()

In [ ]:
## Getting Store visit information from Google Sheet

In [ ]:
## 1st report

In [ ]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

In [ ]:
credentials = ServiceAccountCredentials.from_json_keyfile_name('***Confidential', scope)

In [ ]:
gc = gspread.authorize(credentials)

In [ ]:
worksheet_gio = gc.open('US Field Team Report 2018').get_worksheet(0)
record_gio = worksheet_gio.get_all_records()
df_gio = pd.DataFrame(data=record_gio)
df_gio = df_gio[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_gio = df_gio.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'                                
                                })

for index, row in df_gio.iterrows():
    df_gio.at[index, "Location_Number"] = str(df_gio.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_felix = gc.open('US Field Team Report 2018').get_worksheet(1)
record_felix = worksheet_felix.get_all_records()
df_felix = pd.DataFrame(data=record_felix)
df_felix = df_felix[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_felix = df_felix.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_felix.iterrows():
    df_felix.at[index, "Location_Number"] = str(df_felix.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_freddie = gc.open('US Field Team Report 2018').get_worksheet(2)
record_freddie = worksheet_freddie.get_all_records()
df_freddie = pd.DataFrame(data=record_freddie)
df_freddie = df_freddie[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_freddie = df_freddie.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_freddie.iterrows():
    df_freddie.at[index, "Location_Number"] = str(df_freddie.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_khoa = gc.open('US Field Team Report 2018').get_worksheet(3)
record_khoa = worksheet_khoa.get_all_records()
df_khoa = pd.DataFrame(data=record_khoa)
df_khoa = df_khoa[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_khoa = df_khoa.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_khoa.iterrows():
    df_khoa.at[index, "Location_Number"] = str(df_khoa.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_maggie = gc.open('US Field Team Report 2018').get_worksheet(4)
record_maggie = worksheet_maggie.get_all_records()
df_maggie = pd.DataFrame(data=record_maggie)
df_maggie = df_maggie[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_maggie = df_maggie.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_maggie.iterrows():
    df_maggie.at[index, "Location_Number"] = str(df_maggie.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_jon = gc.open('US Field Team Report 2018').get_worksheet(5)
record_jon = worksheet_jon.get_all_records()
df_jon = pd.DataFrame(data=record_jon)
df_jon = df_jon[['Store#/Name', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_jon = df_jon.rename(columns={'Store#/Name': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_jon.iterrows():
    df_jon.at[index, "Location_Number"] = str(df_jon.at[index, "Location_Number"]).replace("B", "")

In [ ]:
# 2nd report

In [ ]:
worksheet_gio_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(0)
record_gio_2 = worksheet_gio_2.get_all_records()
df_gio_2 = pd.DataFrame(data=record_gio_2)
df_gio_2 = df_gio_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_gio_2 = df_gio_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'                                
                                })

for index, row in df_gio_2.iterrows():
    df_gio_2.at[index, "Location_Number"] = str(df_gio_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_felix_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(1)
record_felix_2 = worksheet_felix_2.get_all_records()
df_felix_2 = pd.DataFrame(data=record_felix_2)
df_felix_2 = df_felix_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_felix_2 = df_felix_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_felix_2.iterrows():
    df_felix_2.at[index, "Location_Number"] = str(df_felix_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_freddie_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(2)
record_freddie_2 = worksheet_freddie_2.get_all_records()
df_freddie_2 = pd.DataFrame(data=record_freddie_2)
df_freddie_2 = df_freddie_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_freddie_2 = df_freddie_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_freddie_2.iterrows():
    df_freddie_2.at[index, "Location_Number"] = str(df_freddie_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_khoa_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(3)
record_khoa_2 = worksheet_khoa_2.get_all_records()
df_khoa_2 = pd.DataFrame(data=record_khoa_2)
df_khoa_2 = df_khoa_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_khoa_2 = df_khoa_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_khoa_2.iterrows():
    df_khoa_2.at[index, "Location_Number"] = str(df_khoa_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_maggie_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(4)
record_maggie_2 = worksheet_maggie_2.get_all_records()
df_maggie_2 = pd.DataFrame(data=record_maggie_2)
df_maggie_2 = df_maggie_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_maggie_2 = df_maggie_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_maggie_2.iterrows():
    df_maggie_2.at[index, "Location_Number"] = str(df_maggie_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
worksheet_jon_2 = gc.open('US_Field_Team_Report_2019_Feb-').get_worksheet(5)
record_jon_2 = worksheet_jon_2.get_all_records()
df_jon_2 = pd.DataFrame(data=record_jon_2)
df_jon_2 = df_jon_2[['Store#', 'Visit Date', 'Who visited', 'Status(P/C)', 'Photo 1', 'Photo 2', 'Photo 3', 'Photo 4', 'Photo 5', 'Key Findings / Pending issues / Status update', 'Memo']]
df_jon_2 = df_jon_2.rename(columns={'Store#': 'Location_Number', 
                                'Visit Date': 'Last_Visit',
                                'Who visited': 'Who_Visited',
                                'Status(P/C)': 'Status',
                                'Photo 1': 'Photo_1',
                                'Photo 2': 'Photo_2',
                                'Photo 3': 'Photo_3',
                                'Photo 4': 'Photo_4',
                                'Photo 5': 'Photo_5',
                                'Key Findings / Pending issues / Status update': 'Comment',
                                'Memo': 'Memo'  
                                })

for index, row in df_jon_2.iterrows():
    df_jon_2.at[index, "Location_Number"] = str(df_jon_2.at[index, "Location_Number"]).replace("B", "")

In [ ]:
#Merging each df
frames = [df_gio, df_felix, df_freddie, df_khoa, df_maggie, df_jon, df_gio_2, df_felix_2, df_freddie_2, df_khoa_2, df_maggie_2, df_jon_2]
merged_store_visit_df = pd.concat(frames)

In [ ]:
merged_store_visit_df

In [ ]:
#Convert date into datetime data type
merged_store_visit_df['Last_Visit'] = pd.to_datetime(merged_store_visit_df.Last_Visit)

In [ ]:
#Sorting by date (new to old)
merged_store_visit_df = merged_store_visit_df.sort_values('Last_Visit', ascending=False)

In [ ]:
#Resetting index
merged_store_visit_df = merged_store_visit_df.reset_index(drop=True)

In [ ]:
merged_store_visit_df

In [ ]:
#Storing "memo" in separate dictionary
memo_dict = {}

for index, row in merged_store_visit_df.iterrows():
    if row['Memo'] != '':
        if row['Location_Number'] not in list(memo_dict.keys()):
            memo_dict[str(row['Location_Number'])] = str(row['Memo']) + ", "
        else:
            memo_dict[str(row['Location_Number'])] += str(row['Memo']) + ", "

In [ ]:
memo_dict

In [ ]:
# Making memo_df
memo_df = pd.DataFrame(list(memo_dict.items()), columns=['Location_Number', 'Memo'])
memo_df.head()

In [ ]:
merged_store_visit_df.head()

In [ ]:
# Dropping "Memo" from merged_store_visit_df
merged_store_visit_df = merged_store_visit_df.drop(columns=['Memo'])
merged_store_visit_df.head()

In [ ]:
#Picking up the latest visit (Replace Location_Number to "Already_Counted" if it is once counted.)
store_list = []

for index, row in merged_store_visit_df.iterrows():
    if row['Location_Number'] in store_list:
        merged_store_visit_df.at[index, 'Location_Number'] = "Already_Counted"
    else:
        store_list.append(row['Location_Number'])     

In [ ]:
#Left merge 1 with Geo data
BBY_Store_List_GeoData_Sell_thru_Store_visit_df = BBY_Store_List_GeoData_Sell_thru_df.merge(merged_store_visit_df, on='Location_Number', how='left')

In [ ]:
#Left merge 2 with Memo data
BBY_Store_List_GeoData_Sell_thru_Store_visit_df = BBY_Store_List_GeoData_Sell_thru_Store_visit_df.merge(memo_df, on='Location_Number', how='left')

In [ ]:
BBY_Store_List_GeoData_Sell_thru_Store_visit_df.head()

In [ ]:
BBY_Store_List_GeoData_Sell_thru_Store_visit_df.to_csv('Resources/for_sell_thru_update/geodata_sell_thru_visit.csv', index=False)

In [ ]:
## Creating JSON file from the geodata_sell_thru_visit.csv

In [ ]:
file_path = "Resources/for_sell_thru_update/geodata_sell_thru_visit.csv"

In [ ]:
features = []
with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for latitude, longitude, location_number, location_name, address_1, address_2, city, state, zip_code, \
    telephone_nbr, current_endcap, current_table_display, t1_update, last_week_st, last_5wk_ave_st, last_10wk_ave_st, \
    last_20wk_ave_st, last_wk_share, last_5wk_share, last_10wk_share, last_20wk_share, last_5wk_performance, lastwknumber, premium_10wk, premium_10wk_share, \
    last_visit, who_visited, status, photo_1, photo_2, photo_3, photo_4, photo_5, comment, memo in reader:
        latitude, longitude = map(float, (latitude, longitude))
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'storenumber': location_number,
                    'storename': location_name,
                    'address1': address_1,
                    'address2': address_2,
                    'city': city,
                    'state': state,
                    'zipcode': zip_code,
                    'telephonenumber': telephone_nbr,
                    'endcap': current_endcap,
                    'tabledisplay': current_table_display,
                    't1update': t1_update,
                    'lastwkst': last_week_st,
                    'last5wkavest': last_5wk_ave_st,
                    'last10wkavest': last_10wk_ave_st,
                    'last20wkavest': last_20wk_ave_st,
                    'lastwkshare': last_wk_share,
                    'last5wkshare': last_5wk_share,
                    'last10wkshare': last_10wk_share,
                    'last20wkshare': last_20wk_share,
                    'last5wkperformance': last_5wk_performance,
                    'lastwknumber': lastwknumber,
                    'premium_10wk': premium_10wk,
                    'premium_10wk_share': premium_10wk_share,
                    'last_visit': last_visit,
                    'who_visited': who_visited,
                    'status': status,
                    'photo_1': photo_1,
                    'photo_2': photo_2,
                    'photo_3': photo_3,
                    'photo_4': photo_4,
                    'photo_5': photo_5,                    
                    'comment': comment,
                    'memo': memo
                }
            )
        )

collection = FeatureCollection(features)
with open("Resources/GeoJson_Data_BBY.json", "w") as f:
    f.write('%s' % collection)

In [ ]:
collection

In [ ]:
len(collection["features"])

In [ ]:
## Deleting previous JSON file from AWS S3 and Uploading new JSON file to AWS S3

In [ ]:
# ID & Secret Key created on 12/21/2018 
s3_id = '***Confidential'
s3_secret = '***Confidential'

In [ ]:
# Connecting to AWS S3 and making an object of the previous JSON file
bucket = "***Confidential"
file_name = "GeoJson_Data_BBY.json"

s3client = boto3.resource('s3', aws_access_key_id=s3_id, aws_secret_access_key=s3_secret)

content_object = s3client.Object(bucket, file_name)

In [ ]:
# Deleting the previous JSON file from AWS S3 Bucket
content_object.delete()

In [ ]:
# Adding New JSON file to AWS S3 Bucket
s3client.Object(bucket, file_name).upload_file(Filename='Resources/GeoJson_Data_BBY.json')